# Assignment 1

In [2]:
from collections import defaultdict, deque
import itertools
import numpy as np
import random
from enum import Enum
from typing import List, Callable, Any, Tuple
from dataclasses import dataclass
import time

In [56]:
class Graph:
    def __init__(self, U_size, V_size):
        self.U = list(range(1, U_size + 1))  # Nodes in fixed layer U
        self.V = list(range(U_size + 1, U_size + V_size + 1))  # Nodes in V
        self.edges = []  # List of edges (u, v, weight)
        self.constraints = defaultdict(list)  # Constraints as adjacency list for V
        self.in_degree = {v: 0 for v in self.V}   # Dictionary to store in-degree of nodes in V
        self.node_edges = defaultdict(list)  # Edges connected to each node

    def add_node_U(self, node):
        self.U.append(node)

    def add_node_V(self, node):
        self.V.append(node)
        self.in_degree[node] = 0  # Initialize in-degree for nodes in V

    def add_edge(self, u, v, weight):
        self.edges.append((u, v, weight))
        self.node_edges[v].append((u, weight))

    def add_constraint(self, v1, v2):
        self.constraints[v1].append(v2)
        self.in_degree[v2] += 1  # Update in-degree due to precedence constraint

In [4]:
def load_instance(filename):
    with open(filename, 'r') as file:
        U_size, V_size, C_size, E_size = map(int, file.readline().split())
        graph = Graph(U_size, V_size)

        # Read constraints section
        line = file.readline().strip()
        while line != "#constraints":
            line = file.readline().strip()

        for _ in range(C_size):
            v, v_prime = map(int, file.readline().split())
            graph.add_constraint(v, v_prime)

        # Read edges section
        line = file.readline().strip()
        while line != "#edges":
            line = file.readline().strip()

        for _ in range(E_size):
            u, v, weight = file.readline().split()
            graph.add_edge(int(u), int(v), float(weight))

    return graph

In [5]:
def cost_function(graph, permutation):
    # Create a dictionary for quick lookup of node positions in the current ordering
    position = {node: idx for idx, node in enumerate(permutation)}
    total_cost = 0
    # Iterate over all pairs of edges to count crossings
    for (u1, v1, w1), (u2, v2, w2) in itertools.combinations(graph.edges, 2):
        # Check if edges cross based on their positions
        if (u1 < u2 and position[v1] > position[v2]) or (u1 > u2 and position[v1] < position[v2]):
            # Add the sum of weights for the crossing edges to the total cost
            total_cost += w1 + w2

    return total_cost

## Question 1: Think about a meaningful real-world application for this problem and briefly describe it

Application: Railway Scheduling and Track Layout

- Nodes in $U$ could represent fixed train stations along a route, where trains must stop in a specific order
- Nodes in $V$ could reoresent trains/train routes to be scheduled within the network
- Weighetd edges would inidcate the relationship between trains/train routes and tracks, with higher weights representing either higher traffic, longer distance, lower importance in terms of scheduling (in case of for exaple local vs. high speed train)
- constraints in C would enforce rules like the fact that certain trains need to arrive before others or that trains using the same tracks do not overlap.

The objective then would be to arrange the schedules in V to minimize the crossings of tracks while satisfying all contraints in C. This would lead to improved security and efficiency as minimzing track crossings would reduce the likelihood of collisions or delaysdue to track conflicts.

## Question 2: Develop a meaningful deterministic construction heuristic

A meaningful determinitic contruction heuristic could be one based on a greedy approach:
- Inititalize an empty ordered permuation list $\pi$ for nodes in $V$
- Compute for each node in $V$ the number of constraints that require other nodes to come before it
- Create a list of candidates with nodes in $V$ that have no predecessors

- For every node in the candidates and until $V$ is empty
  - calculate the total weight of edges connecting it to U
  - select the node with the lowest total edge weight to nodes in $U$ and append it to the final permutation list $\pi$

  - update the in-degree of nodes that had this as a constraint reducing it by 1

  - if the in-degree of any node in V reaches zero add it to the candidates




In [6]:
class DeterministicConstruction:
    def __init__(self, graph):
        self.graph = graph
        self.pi = []  # store the final order of nodes in V

        # Precompute edge weights per node for efficiency
        self.node_weights = defaultdict(int)
        for u, v, weight in graph.edges:
            self.node_weights[v] += weight

    def greedy_construction(self):
        # Initialize candidates with in-degree 0 nodes
        candidates = deque([v for v in self.graph.V if self.graph.in_degree[v] == 0])

        while candidates:
            # Select node with lowest total edge weight
            best_node = min(candidates, key=lambda v: self.node_weights[v])

            # Add the selected best_node to the pi
            self.pi.append(best_node)
            candidates.remove(best_node)

            # Update in-degrees and add new candidates
            for v_next in self.graph.constraints[best_node]:
                self.graph.in_degree[v_next] -= 1
                if self.graph.in_degree[v_next] == 0:
                    candidates.append(v_next)
        # Verify the solution before returning
        if not self.verify_solution():
            raise ValueError("Construction resulted in invalid solution!")

        return self.pi

    def calculate_cost(self):
        # Create a dictionary for quick lookup of node positions
        position = {node: idx for idx, node in enumerate(self.pi)}
        total_cost = 0

        # Calculate crossings and their costs
        for (u1, v1, w1), (u2, v2, w2) in itertools.combinations(self.graph.edges, 2):
            if (u1 < u2 and position[v1] > position[v2]) or \
               (u1 > u2 and position[v1] < position[v2]):
                total_cost += w1 + w2

        return total_cost

    def verify_solution(self):
        """
        Verify that the solution respects all constraints
        """
        if not self.pi:
            return False

        # Check if all nodes from V are present
        if set(self.pi) != set(self.graph.V):
            return False

        # Check if constraints are respected
        position = {node: idx for idx, node in enumerate(self.pi)}
        for v1 in self.graph.V:
            for v2 in self.graph.constraints[v1]:
                if position[v1] > position[v2]:
                    return False

        return True

In [7]:
import os

def list_files_in_folder(folder_path):
  """Lists files and folders within a given directory."""
  items = []
  try:
    for item in os.listdir(folder_path):
      item_path = os.path.join(folder_path, item)
      items.append(item_path)
    return items
  except FileNotFoundError:
    print(f"Folder not found: {folder_path}")

folder_path = "/content/competition_instances"
items = list_files_in_folder(folder_path)

In [8]:
for item in items:
    graph = load_instance(item)
    solution = DeterministicConstruction(graph)
    ordering = solution.greedy_construction()

    instance_name = os.path.basename(item)
    with open(f"{instance_name}.txt", "w") as f:
        f.write(instance_name + "\n")
        f.write(" ".join(map(str, ordering)) + "\n")

In [9]:
# Example usage with a small problem instance
graph = Graph(5, 5)

# Adding edges with weights
graph.add_edge(1, 7, 1)
graph.add_edge(2, 8, 2)
graph.add_edge(3, 9, 1)
graph.add_edge(4, 10, 3)
graph.add_edge(5, 6, 11)

# Adding precedence constraints
graph.add_constraint(7, 10)
graph.add_constraint(8, 9)

# Create an MWCCP solution instance and use the greedy heuristic
solution = DeterministicConstruction(graph)
ordering = solution.greedy_construction()


print("The initial ordering of nodes in V:", ordering)

print(solution.calculate_cost())

The initial ordering of nodes in V: [7, 8, 9, 10, 6]
0


In [10]:
permutation = np.random.permutation(graph.V)
print(permutation)

obj_cost = cost_function(graph, permutation)
print(obj_cost)

obj_cost = cost_function(graph, ordering)
print(obj_cost)

[ 8  7 10  6  9]
19
0


In [11]:
def load_and_order_instance(filename):
    graph = load_instance(filename)
    solution = DeterministicConstruction(graph)
    ordering = solution.greedy_construction()
    return graph, ordering

In [12]:
graph = load_instance('in.txt')

solution = DeterministicConstruction(graph)
ordering = solution.greedy_construction()

print("The initial ordering of nodes in V:", ordering)

print(solution.calculate_cost())


The initial ordering of nodes in V: [35, 43, 39, 48, 27, 49, 26, 31, 30, 34, 37, 38, 45, 46, 28, 40, 33, 41, 32, 42, 36, 47, 50, 29, 44]
94914.0


## Question 3: Derive a randomized construction heuristic to be applied iteratively

The randomized heuristic could follow the same process as the deterministic one, but then, when choosing the node in $V$ to add to the final permutation list $\pi$, instead of always choosing the one that has a minimum weighted sum of edges to $U$, pick one randomly, with a probability inversely propotional to this sum.

In [13]:
class RandomizedConstruction:
    def __init__(self, graph):
        self.graph = graph
        self.pi = []  # Final order of nodes in V

        # Precompute edge weights per node for efficiency
        self.node_weights = defaultdict(int)
        self.node_connections = defaultdict(list)
        for u, v, weight in graph.edges:
            self.node_weights[v] += weight
            self.node_connections[v].append((u, weight))

        # Parameters for randomization
        self.alpha = 0.4

    def calculate_node_score(self, node, current_ordering):
        """
        Calculate a score for a node based on both its weights and potential crossings
        """
        if not current_ordering:
            return self.node_weights[node]

        score = 0
        position = len(current_ordering)

        # Consider existing edges and potential crossings
        for u, weight in self.node_connections[node]:
            score += weight  # Base weight

            # Check potential crossings with already placed nodes
            #for prev_node in current_ordering:
            #    for prev_u, prev_weight in self.node_connections[prev_node]:
            #        if (u < prev_u and position > current_ordering.index(prev_node)) or \
            #           (u > prev_u and position < current_ordering.index(prev_node)):
            #            score += (weight + prev_weight) * 0.5  # Penalty for potential crossing

        return score

    def calculate_probabilities(self, candidates, current_ordering):
        """
        Calculate selection probabilities using Boltzmann distribution
        """
        scores = []
        for node in candidates:
            score = self.calculate_node_score(node, current_ordering)
            scores.append(score)

        scores = np.array(scores)

        # Convert scores to probabilities (lower scores = higher probability)
        max_score = max(scores) if scores.size > 0 else 1
        normalized_scores = scores / max_score  # Normalize to prevent overflow

        # Apply Boltzmann distribution with temperature
        probs = np.exp(-normalized_scores / self.alpha)
        probs = probs / np.sum(probs)

        return probs

    def greedy_randomized_construction(self, num_iterations=1):
        """
        Perform multiple iterations and return the best solution
        """
        best_solution = None
        best_cost = float('inf')

        for _ in range(num_iterations):
            solution = self.construct_single_solution()
            cost = self.calculate_cost(solution)

            if cost < best_cost:
                best_cost = cost
                best_solution = solution.copy()


        self.pi = best_solution
        return best_solution

    def construct_single_solution(self):
        """
        Construct a single solution using randomized greedy approach
        """
        # Reset in-degrees for this iteration
        in_degree = self.graph.in_degree.copy()
        current_ordering = []

        # Initialize candidates with in-degree 0 nodes
        candidates = deque([v for v in self.graph.V if in_degree[v] == 0])

        while candidates:
            candidates_list = list(candidates)

            if len(candidates_list) == 1:
                # If only one candidate, select it directly
                selected_node = candidates_list[0]
            else:
                # Calculate probabilities and select node
                probs = self.calculate_probabilities(candidates_list, current_ordering)
                selected_node = np.random.choice(candidates_list, p=probs)

            current_ordering.append(selected_node)
            candidates.remove(selected_node)

            # Update in-degrees and add new candidates
            for v_next in self.graph.constraints[selected_node]:
                in_degree[v_next] -= 1
                if in_degree[v_next] == 0:
                    candidates.append(v_next)

        return current_ordering

    def calculate_cost(self, ordering=None):
        """
        Calculate the cost of a given ordering or the current pi
        """
        if ordering is None:
            ordering = self.pi

        position = {node: idx for idx, node in enumerate(ordering)}
        total_cost = 0

        for (u1, v1, w1), (u2, v2, w2) in itertools.combinations(self.graph.edges, 2):
            if (u1 < u2 and position[v1] > position[v2]) or \
               (u1 > u2 and position[v1] < position[v2]):
                total_cost += w1 + w2

        return total_cost

    def verify_solution(self):
        """
        Verify that the solution respects all constraints
        """
        if not self.pi:
            return False

        # Check if all nodes from V are present
        if set(self.pi) != set(self.graph.V):
            return False

        # Check if constraints are respected
        position = {node: idx for idx, node in enumerate(self.pi)}
        for v1 in self.graph.V:
            for v2 in self.graph.constraints[v1]:
                if position[v1] > position[v2]:
                    return False

        return True

In [14]:
graph = Graph(5, 5)

# Adding edges with weights
graph.add_edge(1, 7, 1)
graph.add_edge(2, 8, 2)
graph.add_edge(3, 9, 1)
graph.add_edge(4, 10, 3)
graph.add_edge(5, 6, 11)

# Adding precedence constraints
graph.add_constraint(7, 10)
graph.add_constraint(8, 9)


solution = RandomizedConstruction(graph)
ordering = solution.greedy_randomized_construction()


print("The initial ordering of nodes in V:", ordering)

print(solution.calculate_cost())

The initial ordering of nodes in V: [8, 9, 7, 10, 6]
5


In [15]:
graph, ordering = load_and_order_instance('in.txt')

print("The initial ordering of nodes in V:", ordering)

print(solution.calculate_cost())


The initial ordering of nodes in V: [35, 43, 39, 48, 27, 49, 26, 31, 30, 34, 37, 38, 45, 46, 28, 40, 33, 41, 32, 42, 36, 47, 50, 29, 44]
5


In [16]:
for item in items:
    graph, ordering = load_and_order_instance(item)

    instance_name = os.path.basename(item)
    with open(f"{instance_name}.txt", "w") as f:
        f.write(instance_name + "\n")
        f.write(" ".join(map(str, ordering)) + "\n")

## 4. Develop or make use of a framework for basic local search which is able to deal with: different neighborhood structures and different step functions (first-improvement, best-improvement, random)


In [17]:
class LocalSearch:
    def __init__(self, initial_solution, neighborhood_function, step_function, objective_function, max_iter=500):
        """
        Local Search framework.

        Args:
        - initial_solution: Starting solution for the search
        - neighborhood_function: Function to generate neighbor solutions
        - step_function: Function to select next step in search
        - objective_function: Function to compute the cost of a solution
        - max_iter: Maximum number of iterations for the search
        """
        self.current_solution = initial_solution
        self.neighborhood_function = neighborhood_function
        self.step_function = step_function
        self.objective_function = objective_function
        self.max_iter = max_iter

    def local_search(self):
          best_solution = self.current_solution
          best_cost = self.objective_function(best_solution)

          for i in range(self.max_iter):

            neighbors = self.neighborhood_function(self.current_solution)
            next_solution = self.step_function(neighbors, self.objective_function)
            next_cost = self.objective_function(next_solution)

            if next_cost < best_cost:
              best_solution = next_solution
              best_cost = next_cost
              continue


            if best_cost == 0:
              break

            self.current_solution = next_solution


          print("Required iterations:", i)
          return best_solution, best_cost


def best_improvement(neighbors, objective_function):
    if not neighbors:
      return None

    best_neighbor = None
    best_cost = float('inf')

    for neighbor in neighbors:
      cost = objective_function(neighbor)
      if cost < best_cost:
        best_neighbor = neighbor
        best_cost = cost

    return best_neighbor


def first_improvement(neighbors, objective_function):
    for neighbor in neighbors:
      if objective_function(neighbor) < objective_function(neighbors[0]):
        return neighbor
    return neighbors[0]


def random_neighbor(neighbors, objective_function):
  return random.choice(neighbors)


def cost_function(graph, permutation): # same function as above
    if permutation is None:
        return float('inf')
    # Create a dictionary for quick lookup of node positions in the current ordering
    position = {node: idx for idx, node in enumerate(permutation)}
    total_cost = 0
    # Iterate over all pairs of edges to count crossings
    for (u1, v1, w1), (u2, v2, w2) in itertools.combinations(graph.edges, 2):
        # Check if edges cross based on their positions
        if (u1 < u2 and position[v1] > position[v2]) or (u1 > u2 and position[v1] < position[v2]):
            # Add the sum of weights for the crossing edges to the total cost
            total_cost += w1 + w2

    return total_cost

## 5. Develop at least three different meaningful neighborhood structures

In [18]:
def is_valid_operator(solution, constraints):
    position = {node: idx for idx, node in enumerate(solution)}
    for first, second in constraints:
      if position[second] < position[first]:
        return False
    return True

### 1. Swap operator

In [19]:
def swap_neighborhood(solution, constraints):
    if solution is None:
        return []

    neighbors = []
    if not isinstance(solution, list):
        solution = list(solution)

    for i in range(len(solution) - 1):
        neighbor = solution.copy()
        neighbor[i], neighbor[i + 1] = neighbor[i + 1], neighbor[i]
        if is_valid_operator(neighbor, constraints) and neighbor not in neighbors:
            neighbors.append(neighbor)

    return neighbors

In [20]:
#initial_solution = np.random.permutation(graph.V)
#initial_solution = ordering
initial_solution = [6, 8, 7, 10, 9]

print("The initial ordering of nodes in V:", initial_solution)
print(cost_function(graph, initial_solution))

constraints = [(key, item[0]) for key, item in graph.constraints.items() if len(item)>0]

neighborhood_function = lambda sol: swap_neighborhood(sol, constraints)
objective_function = lambda sol: cost_function(graph, sol)

# Step functions
for step_fun in [best_improvement, first_improvement, random_neighbor]:
    print("----")
    local_search = LocalSearch(initial_solution, neighborhood_function, step_fun, objective_function)
    best_solution, best_cost = local_search.local_search()

    print(step_fun.__name__)
    print("Improvement:", best_solution)
    print(best_cost)

The initial ordering of nodes in V: [6, 8, 7, 10, 9]


KeyError: 255

### 2. Insert operator

In [21]:
def insert_neighborhood(solution, constraints):
    if solution is None:
        return []

    neighbors = set()
    solution = list(solution)

    for i in range(len(solution)):
        for j in range(len(solution)):
            if i != j:
                neighbor = solution.copy()
                neighbor.insert(j, neighbor.pop(i))
                neighbor = tuple(neighbor)
                if is_valid_operator(neighbor, constraints) and neighbor not in neighbors:
                    neighbors.add(neighbor)

    return list(neighbors)

In [22]:
initial_solution = ordering
#initial_solution = np.random.permutation(graph.V)
initial_solution = [6, 8, 7, 10, 9]
print("The initial ordering of nodes in V:", initial_solution)
print(cost_function(graph, initial_solution))

neighborhood_function = lambda sol: insert_neighborhood(sol, constraints)
objective_function = lambda sol: cost_function(graph, sol)

# Improvement step function
for step_fun in [best_improvement, first_improvement, random_neighbor]:
    print("----")
    local_search = LocalSearch(initial_solution, neighborhood_function, step_fun, objective_function)
    best_solution, best_cost = local_search.local_search()

    print(step_fun.__name__)
    print("Improvement:", best_solution)
    print(best_cost)

The initial ordering of nodes in V: [6, 8, 7, 10, 9]


KeyError: 255

### 3. reverse operator

In [23]:
def reverse_neighborhood(solution, constraints):
    if solution is None:
        return []

    neighbors = set()

    for i in range(len(solution)):
        neighbor_first = tuple(solution[:i])
        for j in range(i + 1, len(solution)):
            neighbor = neighbor_first + tuple(reversed(solution[i: j + 1])) + tuple(solution[j + 1:])
            # neighbor[i: j+1] = list(reversed(neighbor[i:j+1]))
            if is_valid_operator(neighbor, constraints) and neighbor not in neighbors:
                neighbors.add(neighbor)
    return list(neighbors)

In [24]:
initial_solution = ordering
initial_solution = [6, 8, 7, 10, 9]

print("The initial ordering of nodes in V:", initial_solution)
print(cost_function(graph, initial_solution))

neighborhood_function = lambda sol: reverse_neighborhood(sol, constraints)
objective_function = lambda sol: cost_function(graph, sol)

for step_fun in [best_improvement, first_improvement, random_neighbor]:
    print("----")
    local_search = LocalSearch(initial_solution, neighborhood_function, step_fun, objective_function)
    best_solution, best_cost = local_search.local_search()

    print(step_fun.__name__)
    print("Improvement:", best_solution)
    print(best_cost)

The initial ordering of nodes in V: [6, 8, 7, 10, 9]


KeyError: 255

## Improved version

In [25]:
class StepFunction(Enum):
    BEST_IMPROVEMENT = "best_improvement"
    FIRST_IMPROVEMENT = "first_improvement"
    RANDOM = "random"

class NeighborhoodType(Enum):
    SWAP = "swap"
    INSERT = "insert"
    REVERSE = "reverse"

@dataclass
class SearchStatistics:
    iterations: int
    runtime: float
    improvement_history: List[float]
    best_cost: float
    plateau_counts: int
    local_optima_count: int

class LocalSearch:
    def __init__(self,
                 graph,
                 initial_solution: List[int],
                 neighborhood_type: NeighborhoodType,
                 step_function: StepFunction,
                 max_iter: int = 500,
                 max_plateau: int = 50,
                 memory_size: int = 10):
        """
        Enhanced Local Search framework.

        Args:
            graph: The graph object containing edges and constraints
            initial_solution: Starting permutation
            neighborhood_type: Type of neighborhood structure to use
            step_function: Method for selecting next solution
            max_iter: Maximum iterations
            max_plateau: Maximum iterations without improvement
            memory_size: Size of solution history to maintain
        """
        self.graph = graph
        self.current_solution = initial_solution.copy()
        self.neighborhood_type = neighborhood_type
        self.step_function = step_function
        self.max_iter = max_iter
        self.max_plateau = max_plateau
        self.memory_size = memory_size
        self.best_solutions = []  # Store best solutions found

        # Initialize statistics
        self.stats = SearchStatistics(
            iterations=0,
            runtime=0,
            improvement_history=[],
            best_cost=float('inf'),
            plateau_counts=0,
            local_optima_count=0
        )

    def verify_constraints(self, solution: List[int]) -> bool:
        """Check if solution respects all constraints"""
        position = {node: idx for idx, node in enumerate(solution)}
        for v1 in self.graph.V:
            for v2 in self.graph.constraints[v1]:
                if position[v1] > position[v2]:
                    return False
        return True

    def swap_neighborhood(self, solution: List[int]) -> List[List[int]]:
        """Generate neighbors by swapping adjacent pairs that respect constraints"""
        neighbors = []
        for i in range(len(solution) - 1):
            neighbor = solution.copy()
            # Only swap if it doesn't violate constraints
            if (neighbor[i] not in self.graph.constraints[neighbor[i+1]] and
                neighbor[i+1] not in self.graph.constraints[neighbor[i]]):
                neighbor[i], neighbor[i+1] = neighbor[i+1], neighbor[i]
                if self.verify_constraints(neighbor):
                    neighbors.append(neighbor)
        return neighbors

    def insert_neighborhood(self, solution: List[int]) -> List[List[int]]:
        """Generate neighbors by inserting elements at different positions"""
        neighbors = []
        for i in range(len(solution)):
            for j in range(len(solution)):
                if i != j:
                    neighbor = solution.copy()
                    element = neighbor.pop(i)
                    neighbor.insert(j, element)
                    if self.verify_constraints(neighbor):
                        neighbors.append(neighbor)
        return neighbors

    def reverse_neighborhood(self, solution: List[int]) -> List[List[int]]:
        """Generate neighbors by reversing subsequences"""
        neighbors = []
        for i in range(len(solution)):
            for j in range(i + 2, len(solution)):
                neighbor = solution.copy()
                neighbor[i:j] = reversed(neighbor[i:j])
                if self.verify_constraints(neighbor):
                    neighbors.append(neighbor)
        return neighbors

    def generate_neighborhood(self, solution: List[int]) -> List[List[int]]:
        """Generate neighbors based on selected neighborhood type"""
        if self.neighborhood_type == NeighborhoodType.SWAP:
            return self.swap_neighborhood(solution)
        elif self.neighborhood_type == NeighborhoodType.INSERT:
            return self.insert_neighborhood(solution)
        elif self.neighborhood_type == NeighborhoodType.REVERSE:
            return self.reverse_neighborhood(solution)


    def calculate_cost(self, solution: List[int]) -> float:
        """Calculate cost of a solution"""
        position = {node: idx for idx, node in enumerate(solution)}
        total_cost = 0

        for (u1, v1, w1), (u2, v2, w2) in itertools.combinations(self.graph.edges, 2):
            if (u1 < u2 and position[v1] > position[v2]) or \
               (u1 > u2 and position[v1] < position[v2]):
                total_cost += w1 + w2

        return total_cost

    def select_next_solution(self, neighbors: List[List[int]]) -> Tuple[List[int], float]:
        """Select next solution based on step function"""
        if not neighbors:
            return self.current_solution, self.calculate_cost(self.current_solution)

        if self.step_function == StepFunction.BEST_IMPROVEMENT:
            best_neighbor = min(neighbors, key=self.calculate_cost)
            return best_neighbor, self.calculate_cost(best_neighbor)

        elif self.step_function == StepFunction.FIRST_IMPROVEMENT:
            current_cost = self.calculate_cost(self.current_solution)
            for neighbor in neighbors:
                neighbor_cost = self.calculate_cost(neighbor)
                if neighbor_cost < current_cost:
                    return neighbor, neighbor_cost
            return self.current_solution, current_cost

        elif self.step_function == StepFunction.RANDOM:
            selected = random.choice(neighbors)
            return selected, self.calculate_cost(selected)

    def update_statistics(self, iteration: int, current_cost: float, is_improvement: bool):
        """Update search statistics"""
        self.stats.iterations = iteration
        self.stats.improvement_history.append(current_cost)

        if is_improvement:
            self.stats.plateau_counts = 0
        else:
            self.stats.plateau_counts += 1
            if self.stats.plateau_counts >= self.max_plateau:
                self.stats.local_optima_count += 1

    def local_search(self) -> Tuple[List[int], float, SearchStatistics]:
        """Execute local search"""
        start_time = time.time()

        best_solution = self.current_solution.copy()
        best_cost = self.calculate_cost(best_solution)
        plateau_counter = 0

        for iteration in range(self.max_iter):
            # Generate neighborhood
            neighbors = self.generate_neighborhood(self.current_solution)

            # Select next solution
            next_solution, next_cost = self.select_next_solution(neighbors)

            # Update best solution if improvement found
            if next_cost < best_cost:
                best_solution = next_solution.copy()
                best_cost = next_cost
                plateau_counter = 0
                self.best_solutions.append((best_solution.copy(), best_cost))
                if len(self.best_solutions) > self.memory_size:
                    self.best_solutions.pop(0)
            else:
                plateau_counter += 1

            # Update statistics
            self.update_statistics(iteration, next_cost, next_cost < best_cost)

            # Check stopping conditions
            if plateau_counter >= self.max_plateau:
                break

            self.current_solution = next_solution

        self.stats.runtime = time.time() - start_time
        return best_solution, best_cost, self.stats

    def get_search_statistics(self) -> SearchStatistics:
        """Return search statistics"""
        return self.stats

In [26]:
graph, ordering = load_and_order_instance('in.txt')
ls = LocalSearch(
    graph=graph,
    initial_solution=ordering,
    neighborhood_type=NeighborhoodType.SWAP,
    step_function=StepFunction.RANDOM,
    max_iter=500,
    max_plateau=50
)

best_solution, best_cost, stats = ls.local_search()

# Print results
print(f"Best cost: {best_cost}")
print(f"Runtime: {stats.runtime:.2f} seconds")
print(f"Iterations: {stats.iterations}")
print(f"Local optima encountered: {stats.local_optima_count}")
print(best_solution)

Best cost: 92390.0
Runtime: 0.74 seconds
Iterations: 267
Local optima encountered: 219
[26, 39, 40, 49, 27, 30, 37, 43, 35, 31, 45, 48, 38, 28, 33, 46, 29, 34, 41, 36, 50, 42, 44, 47, 32]


## 6. Develop or make use of a Variable Neighborhood Descent (VND) framework which uses your neighborhood structures.

In [27]:
class VND:
    def __init__(self, initial_solution, neighborhood_structures, constraints, objective_function, step_function,
                 max_iter=500, verbose=True):
        """
        Variable Neighborhood Descent framework.

        Args:
        - initial_solution: Starting solution for the search
        - neighborhood_structures: List of neighborhood functions to use
        - constraints: List of constraints for the problem
        - objective_function: Function to compute the cost of a solution
        - step_function: Function to select next step in search
        - max_iter: Maximum number of iterations for the search
        """
        self.current_solution = initial_solution
        self.neighborhood_structures = neighborhood_structures
        self.objective_function = objective_function
        self.step_function = step_function
        self.max_iter = max_iter
        self.constraints = constraints
        self.verbose = verbose


    def vnd(self, solution=None):
        if solution is not None:
            self.current_solution = solution
        best_solution = self.current_solution
        best_cost = self.objective_function(best_solution)
        l = 0

        for i in range(self.max_iter):

            # select neighborhood structure l
            neighborhood_function = self.neighborhood_structures[l]


            neighbors = neighborhood_function(self.current_solution, constraints)
            next_solution = self.step_function(neighbors, self.objective_function) # find local optimum in neighborhood

            if next_solution is not None:
                next_cost = self.objective_function(next_solution)
                print(next_cost)

                if next_cost < best_cost:
                    self.current_solution = best_solution
                    best_solution = next_solution
                    best_cost = next_cost
                    l = 0
                else:
                    l += 1

            if l >= len(self.neighborhood_structures):
                break

            if best_cost == 0:
                break

        if self.verbose:
            print("Required iterations for VND:", i + 1)
        return best_solution, best_cost

In [28]:
neighborhood_structures = [swap_neighborhood, insert_neighborhood, reverse_neighborhood]

initial_solution = [6, 8, 7, 10, 9]
initial_solution = ordering
constraints = [(key, item[0]) for key, item in graph.constraints.items() if len(item)>0]

print("The initial ordering of nodes in V:", initial_solution)
print(cost_function(graph, initial_solution))

objective_function = lambda sol: cost_function(graph, sol)

variable_neigh_descent = VND(initial_solution, neighborhood_structures, constraints, objective_function, best_improvement)

best_solution, best_cost = variable_neigh_descent.vnd()

print("Improvement:", best_solution)
print(best_cost)

The initial ordering of nodes in V: [35, 43, 39, 48, 27, 49, 26, 31, 30, 34, 37, 38, 45, 46, 28, 40, 33, 41, 32, 42, 36, 47, 50, 29, 44]
94914.0
94704.0
94704.0
92324.0
94412.0
92114.0
92114.0
90154.0
91939.0
89944.0
89944.0
88641.0
89780.0
88431.0
88431.0
87270.0
88267.0
87060.0
87060.0
86014.0
86896.0
85804.0
85804.0
84880.0
85640.0
84670.0
84670.0
83841.0
84513.0
83631.0
83631.0
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/jurahostic/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/9v/ngk5y1xx5rx4d_zqj33qpjd80000gn/T/ipykernel_1832/2483958244.py", line 14, in <module>
    best_solution, best_cost = variable_neigh_descent.vnd()
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/9v/ngk5y1xx5rx4d_zqj33qpjd80000gn/T/ipykernel_1832/3381980652.py", line 38, in vnd
    next_solution = self.step_function(neighbors, self.objective_function) # find local optimum in neighborhood
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/9v/ngk5y1xx5rx4d_zqj33qpjd80000gn/T/ipykernel_1832/3821553960.py", line 53, in best_improvement
    cost = objective_function(neighbor)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/9v/ngk5y1xx5rx4d_zqj33qpjd80000gn/T/ipykernel_1832

In [39]:
## Improved version
import time
from enum import Enum
from typing import List, Tuple, Callable, Any
from dataclasses import dataclass
import random
import itertools

class NeighborhoodType(Enum):
    SWAP = "swap"
    INSERT = "insert"
    REVERSE = "reverse"

class StepFunction(Enum):
    BEST_IMPROVEMENT = "best_improvement"
    FIRST_IMPROVEMENT = "first_improvement"
    RANDOM = "random"

@dataclass
class VNDStatistics:
    total_iterations: int
    iterations_per_neighborhood: dict
    improvements_per_neighborhood: dict
    runtime: float
    improvement_history: List[float]
    neighborhood_switches: int
    best_cost_history: List[float]
    time_per_neighborhood: dict

class ImprovedVND:
    def __init__(self,
                 graph,
                 initial_solution: List[int],
                 step_function: StepFunction = StepFunction.BEST_IMPROVEMENT,
                 max_iter: int = 100,
                 max_no_improve: int = 20,
                 neighborhood_order: List[NeighborhoodType] = None):
        """
        Enhanced Variable Neighborhood Descent framework.

        Args:
            graph: The graph object containing edges and constraints
            initial_solution: Starting permutation
            step_function: Method for selecting next solution
            max_iter: Maximum total iterations
            max_no_improve: Maximum iterations without improvement
            neighborhood_order: Custom order of neighborhood structures
        """
        self.graph = graph
        self.current_solution = initial_solution.copy()
        self.step_function = step_function
        self.max_iter = max_iter
        self.max_no_improve = max_no_improve
        self.current_cost = self.calculate_cost(self.current_solution)

        # Initialize neighborhood structures if not provided
        self.neighborhood_order = neighborhood_order or [
            NeighborhoodType.SWAP,
            NeighborhoodType.INSERT,
            NeighborhoodType.REVERSE,
        ]

        # Initialize statistics
        self.stats = VNDStatistics(
            total_iterations=0,
            iterations_per_neighborhood={n: 0 for n in self.neighborhood_order},
            improvements_per_neighborhood={n: 0 for n in self.neighborhood_order},
            runtime=0,
            improvement_history=[],
            neighborhood_switches=0,
            best_cost_history=[],
            time_per_neighborhood={n: 0 for n in self.neighborhood_order}
        )

    def swap_neighborhood(self, solution: List[int]) -> List[List[int]]:
        """Generate neighbors by swapping adjacent pairs that respect constraints"""
        neighbors = []
        for i in range(len(solution) - 1):
            neighbor = solution.copy()
            # Only swap if it doesn't violate constraints
            if (neighbor[i] not in self.graph.constraints[neighbor[i+1]] and
                neighbor[i+1] not in self.graph.constraints[neighbor[i]]):
                neighbor[i], neighbor[i+1] = neighbor[i+1], neighbor[i]
                if self.verify_constraints(neighbor):
                    neighbors.append(neighbor)
        return neighbors

    def insert_neighborhood(self, solution: List[int]) -> List[List[int]]:
        """Generate neighbors by inserting elements at different positions"""
        neighbors = []
        for i in range(len(solution)):
            for j in range(len(solution)):
                if i != j:
                    neighbor = solution.copy()
                    element = neighbor.pop(i)
                    neighbor.insert(j, element)
                    if self.verify_constraints(neighbor):
                        neighbors.append(neighbor)
        return neighbors

    def reverse_neighborhood(self, solution: List[int]) -> List[List[int]]:
        """Generate neighbors by reversing subsequences"""
        neighbors = []
        for i in range(len(solution)):
            for j in range(i + 2, len(solution)):
                neighbor = solution.copy()
                neighbor[i:j] = reversed(neighbor[i:j])
                if self.verify_constraints(neighbor):
                    neighbors.append(neighbor)
        return neighbors


    def generate_neighborhood(self, solution: List[int], neighborhood_type: NeighborhoodType) -> List[List[int]]:
        """Generate neighbors based on neighborhood type"""
        if neighborhood_type == NeighborhoodType.SWAP:
            return self.swap_neighborhood(solution)
        elif neighborhood_type == NeighborhoodType.INSERT:
            return self.insert_neighborhood(solution)
        elif neighborhood_type == NeighborhoodType.REVERSE:
            return self.reverse_neighborhood(solution)

    def verify_constraints(self, solution: List[int]) -> bool:
        """Check if solution respects all constraints"""
        position = {node: idx for idx, node in enumerate(solution)}
        for v1 in self.graph.V:
            for v2 in self.graph.constraints[v1]:
                if position[v1] > position[v2]:
                    return False
        return True

    def calculate_cost(self, solution: List[int]) -> float:
        """Calculate cost of a solution"""
        position = {node: idx for idx, node in enumerate(solution)}
        total_cost = 0
        for (u1, v1, w1), (u2, v2, w2) in itertools.combinations(self.graph.edges, 2):
            if (u1 < u2 and position[v1] > position[v2]) or \
               (u1 > u2 and position[v1] < position[v2]):
                total_cost += w1 + w2
        return total_cost

    def select_next_solution(self, neighbors: List[List[int]], current_cost: float) -> Tuple[List[int], float]:
        """Select next solution based on step function"""
        if not neighbors:
            return self.current_solution, current_cost

        if self.step_function == StepFunction.BEST_IMPROVEMENT:
            best_neighbor = min(neighbors, key=self.calculate_cost)
            return best_neighbor, self.calculate_cost(best_neighbor)

        elif self.step_function == StepFunction.FIRST_IMPROVEMENT:
            for neighbor in neighbors:
                neighbor_cost = self.calculate_cost(neighbor)
                if neighbor_cost < current_cost:
                    return neighbor, neighbor_cost
            return self.current_solution, current_cost

        elif self.step_function == StepFunction.RANDOM:
            selected = random.choice(neighbors)
            return selected, self.calculate_cost(selected)

    def update_statistics(self, neighborhood_type: NeighborhoodType, improved: bool, runtime: float):
        """Update search statistics"""
        self.stats.iterations_per_neighborhood[neighborhood_type] += 1
        if improved:
            self.stats.improvements_per_neighborhood[neighborhood_type] += 1
        self.stats.time_per_neighborhood[neighborhood_type] += runtime

    def vnd_search(self) -> Tuple[List[int], float, VNDStatistics]:
        """Execute VND search"""
        start_time = time.time()

        best_solution = self.current_solution.copy()
        best_cost = self.current_cost
        self.stats.best_cost_history.append(best_cost)

        no_improve_counter = 0
        l = 0  # neighborhood index

        for iteration in range(self.max_iter):
            self.stats.total_iterations += 1
            neighborhood_start_time = time.time()

            # Get current neighborhood type
            current_neighborhood = self.neighborhood_order[l]

            # Generate and explore neighborhood
            neighbors = self.generate_neighborhood(self.current_solution, current_neighborhood)
            next_solution, next_cost = self.select_next_solution(neighbors, best_cost)

            # Update statistics
            neighborhood_runtime = time.time() - neighborhood_start_time
            improved = next_cost < best_cost
            self.update_statistics(current_neighborhood, improved, neighborhood_runtime)

            if improved:
                self.current_solution = next_solution
                self.current_cost = next_cost
                best_solution = next_solution
                best_cost = next_cost
                self.stats.best_cost_history.append(best_cost)
                self.stats.improvement_history.append(best_cost)
                l = 0  # Reset to first neighborhood
                no_improve_counter = 0
                self.stats.neighborhood_switches += 1
            else:
                l += 1  # Move to next neighborhood
                no_improve_counter += 1

            # Check stopping conditions
            if l >= len(self.neighborhood_order) or no_improve_counter >= self.max_no_improve:
                break

            if best_cost == 0:
                break

        self.stats.runtime = time.time() - start_time
        return best_solution, best_cost, self.stats

    def get_statistics(self) -> dict:
        """Return detailed statistics about the search"""
        return {
            "total_iterations": self.stats.total_iterations,
            "runtime": self.stats.runtime,
            "improvements_per_neighborhood": dict(self.stats.improvements_per_neighborhood),
            "time_per_neighborhood": dict(self.stats.time_per_neighborhood),
            "neighborhood_switches": self.stats.neighborhood_switches,
            "convergence_history": self.stats.best_cost_history
        }

In [40]:
# Create and run VND
graph, ordering = load_and_order_instance('in.txt')

vnd = ImprovedVND(
    graph=graph,
    initial_solution=ordering,
    step_function=StepFunction.BEST_IMPROVEMENT,
    max_iter=400,
    max_no_improve=200,
    neighborhood_order=[
        NeighborhoodType.SWAP,
        NeighborhoodType.INSERT,
        NeighborhoodType.REVERSE
    ]
)

best_solution, best_cost, stats = vnd.vnd_search()

# Print results and statistics
print(f"Best cost: {best_cost}")
print(f"Runtime: {stats.runtime:.2f} seconds")
print(f"Improvements per neighborhood:")
for n, impr in stats.improvements_per_neighborhood.items():
    print(f"  {n.value}: {impr}")

Best cost: 80230.0
Runtime: 20.26 seconds
Improvements per neighborhood:
  swap: 115
  insert: 8
  reverse: 0


## 7. Implement a Greedy Randomized Adaptive Search Procedure (GRASP) using your randomized construction heuristic and an effective neighborhood structure with one step function or (a variant of) your VND. Note that the union of existing neighborhood structures also constitutes a (composite) neighborhood structure.

In [31]:
class GRASP:
    def __init__(self, graph, alpha, max_iterations, neighborhood_structures, objective_function, step_function, constraints, max_iter_vnd=100):
        """
        GRASP framework with randomized construction and local search.

        Args:
        - graph: The input graph structure.
        - alpha: Parameter for controlling greediness in construction phase (0 = purely greedy, 1 = purely random).
        - max_iterations: Maximum number of GRASP iterations.
        - neighborhood_structures: List of neighborhood functions for local search.
        - objective_function: Function to compute the cost of a solution.
        - step_function: Function for selecting the best solution in a neighborhood.
        """
        self.graph = graph
        self.alpha = alpha
        self.max_iterations = max_iterations
        self.neighborhood_structures = neighborhood_structures
        self.objective_function = objective_function
        self.step_function = step_function
        self.constraints = constraints
        self.max_iter_vnd = max_iter_vnd

    def run(self):
        best_solution = None
        best_cost = float('inf')

        for iteration in range(self.max_iterations):
            # Construction Phase: Generate an initial solution
            constructor = RandomizedConstruction(self.graph)
            initial_solution = constructor.greedy_randomized_construction()
            while len(initial_solution) != len(self.graph.V):
                initial_solution = constructor.greedy_randomized_construction()

            # Local Search Phase: Refine the initial solution
            local_search_constructor = VND(initial_solution, self.neighborhood_structures, self.constraints, self.objective_function, self.step_function, self.max_iter_vnd)
            local_search_solution, local_search_cost = local_search_constructor.vnd()


            # Update the best solution if the improved solution is better
            if local_search_cost < best_cost:
                best_solution = local_search_solution
                best_cost = local_search_cost

            print(f"Iteration {iteration + 1}: Cost = {local_search_cost}, Best Cost = {best_cost}")

            if best_cost == 0:
                break

        return best_solution, best_cost


In [ ]:
graph = load_instance('in.txt')
neighborhood_structures = [swap_neighborhood, insert_neighborhood, reverse_neighborhood]
constraints = [(key, item[0]) for key, item in graph.constraints.items() if len(item)>0]
objective_function = lambda sol: cost_function(graph, sol)
step_function = best_improvement

grasp = GRASP(graph, 0.5, 50, neighborhood_structures, objective_function, step_function, constraints)
best_solution, best_cost = grasp.run()

print("Improvement:", best_solution)
print(best_cost)

# 8. Implement one of the following metaheuristics: • General Variable Neighborhood Search (GVNS) on top of your VND • Simulated Annealing (SA) • Tabu Search (TS)

In [32]:
def swap_neighborhood_shake(solution, constraints):
    if solution is None:
        return []

    # Randomly select two nodes to swap
    while True:
        i, j = random.sample(range(len(solution)), 2)
        neighbor = solution.copy() if isinstance(solution, list) else list(solution)
        neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
        if is_valid_operator(neighbor, constraints):
            return neighbor

def n_swap_neighborhood_shake_generator(n=2):
    def shake(solution, constraints):
        if solution is None:
            return []

        neighbor = solution.copy() if isinstance(solution, list) else list(solution)
        for _ in range(n):
            neighbor = swap_neighborhood_shake(neighbor, constraints)
        return neighbor

    return shake

def insert_neighborhood_shake(solution, constraints):
    if solution is None:
        return []

    # Randomly select two nodes to swap
    while True:
        i, j = random.sample(range(len(solution)), 2)
        neighbor = solution.copy() if isinstance(solution, list) else list(solution)
        neighbor.insert(j, neighbor.pop(i))
        if is_valid_operator(neighbor, constraints):
            return neighbor


def reverse_neighborhood_shake(solution, constraints):
    if solution is None:
        return []

    # Randomly select two nodes to swap
    while True:
        i, j = random.sample(range(len(solution)), 2)
        neighbor_first = tuple(solution[:i])
        neighbor = neighbor_first + tuple(reversed(solution[i: j + 1])) + tuple(solution[j + 1:])
        if is_valid_operator(neighbor, constraints):
            return neighbor


In [33]:
class GVNS:
    def __init__(self, initial_solution, shaking_neighbourhood, local_search_neighbourhoods, constraints,
                 objective_function, max_iter=500, verbose=True):
        """
        General Variable Neighborhood Search framework.

        Args:
        - initial_solution: Starting solution for the search
        - shaking_neighbourhood: Neighborhood function for shaking
        - local_search_neighbourhoods: List of neighborhood functions for local search
        - objective_function: Function to compute the cost of a solution
        - max_iter: Maximum number of iterations for the search
        """
        self.current_solution = initial_solution
        self.shaking_neighbourhood = shaking_neighbourhood
        self.local_search_neighbourhoods = local_search_neighbourhoods
        self.objective_function = objective_function
        self.max_iter = max_iter
        self.constraints = constraints
        # self.VND = VND(graph=graph, initial_solution=initial_solution, step_function=best_improvement, max_iter=max_iter)
        self.VND = VND(initial_solution, local_search_neighbourhoods, constraints, objective_function, best_improvement, max_iter, False)
        self.verbose = verbose

    def find_solution(self):
        self.current_solution, best_cost = self.VND.vnd()
        best_solution = self.current_solution

        for i in range(self.max_iter):
            k = 0
            while k < len(self.shaking_neighbourhood):
                # pick random solution from the shaking neighbourhood
                shaken_x = self.shaking_neighbourhood[k](best_solution, self.constraints)
                new_solution, new_cost = self.VND.vnd(shaken_x)

                if new_cost < best_cost:
                    best_solution, best_cost = new_solution, new_cost
                    k = 0
                else:
                    k += 1

            if best_cost == 0:
                break
        if self.verbose:
            print("Required iterations for GVNS:", i + 1)
        return best_solution, best_cost


In [179]:
shaking_neighbourhood = [swap_neighborhood_shake, n_swap_neighborhood_shake_generator(2), n_swap_neighborhood_shake_generator(3), insert_neighborhood_shake, reverse_neighborhood_shake]
local_search_neighbourhoods = [swap_neighborhood, insert_neighborhood, reverse_neighborhood]

graph, ordering = load_and_order_instance('in.txt')
constraints = [(key, item[0]) for key, item in graph.constraints.items() if len(item) > 0]

print("The initial ordering of nodes in V:", ordering)
print(cost_function(graph, ordering))

objective_function = lambda sol: cost_function(graph, sol)

gvns = GVNS(ordering, shaking_neighbourhood, local_search_neighbourhoods, constraints, objective_function, max_iter=50)

best_solution, best_cost = gvns.find_solution()

print("Improvement:", best_solution)
print(best_cost)


The initial ordering of nodes in V: [35, 43, 39, 48, 27, 49, 26, 31, 30, 34, 37, 38, 45, 46, 28, 40, 33, 41, 32, 42, 36, 47, 50, 29, 44]
94914.0
94704.0
94704.0
92324.0
94412.0
92114.0
92114.0
90154.0
91939.0
89944.0
89944.0
88641.0
89780.0
88431.0
88431.0
87270.0
88267.0
87060.0
87060.0
86014.0
86896.0
85804.0
85804.0
84880.0
85640.0
84670.0
84670.0
83841.0
84513.0
83631.0
83631.0
82781.0
83474.0
82571.0
82571.0
82021.0
82414.0
81811.0
81811.0
81328.0
81654.0
81118.0
81118.0
80874.0
81033.0
80874.0
80934.0
Required iterations for VND: 46
80627.0
80627.0
80555.0
80587.0
80481.0
80515.0
80391.0
80441.0
80389.0
80351.0
80362.0
80327.0
80324.0
80301.0
80298.0
80280.0
80277.0
80260.0
80276.0
80240.0
80242.0
80240.0
80242.0
Required iterations for VND: 23
82423.0
82423.0
81114.0
82312.0
81114.0
81981.0
Required iterations for VND: 6
82002.0
82002.0
80907.0
81820.0
80907.0
81744.0
Required iterations for VND: 6
83616.0
83616.0
82074.0
83468.0
82074.0
82426.0
Required iterations for VND: 6
82

Traceback (most recent call last):
  File "/Users/jurahostic/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/9v/ngk5y1xx5rx4d_zqj33qpjd80000gn/T/ipykernel_91952/1131995606.py", line 14, in <module>
    best_solution, best_cost = gvns.find_solution()
                               ^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/9v/ngk5y1xx5rx4d_zqj33qpjd80000gn/T/ipykernel_91952/2624322586.py", line 33, in find_solution
    new_solution, new_cost = self.VND.vnd(shaken_x)
                             ^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/9v/ngk5y1xx5rx4d_zqj33qpjd80000gn/T/ipykernel_91952/3381980652.py", line 38, in vnd
    next_solution = self.step_function(neighbors, self.objective_function) # find local optimum in neighborhood
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/9v/ngk5y1xx5rx4d_zqj33qpjd80000gn/T/ipyker

# 9. Use delta-evaluation. Explain which steps in your algorithm use delta-evaluation and describe why delta-evaluation results in better performance in this step. Are there other elements in your algorithm that could have also benefitted from delta-evaluation? If possible analyze the asymptotic runtime of a step using delta-evaluation, and of a step without delta-evaluation. Is it possible to pre-process an instance and use derived information later more efficiently?

In [64]:
class DeltaImprovedVND(ImprovedVND):
    def calculate_cost(self, solution: List[int]) -> float:
        """ Calculate the cost of a solution using the delta evaluation"""
        current_solution = self.current_solution
        try:
            current_cost = self.current_cost
        except AttributeError:
            position = {node: idx for idx, node in enumerate(solution)}
            total_cost = 0
            for (u1, v1, w1), (u2, v2, w2) in itertools.combinations(self.graph.edges, 2):
                if (u1 < u2 and position[v1] > position[v2]) or \
                   (u1 > u2 and position[v1] < position[v2]):
                    total_cost += w1 + w2
                    
            return total_cost
        
        new_position = {node: idx for idx, node in enumerate(solution)}
        cur_position = {node: idx for idx, node in enumerate(current_solution)}
        
        diff = [node for node in new_position if new_position[node] != cur_position[node]]
        
        delta_cost = 0
        
        # Remove the cost of the nodes at positions which are True in diff
        evaluated = set()
        for node in diff:
            evaluated.add(node)
            # Iterate through all edges connected to the nodes which have changed position
            for edge in self.graph.node_edges[node]:
                u1, w1 = edge
                v1 = node
                # Check the crossing with all other edges
                for other_edge in self.graph.edges:
                    u2, v2, w2 = other_edge
                    if v2 in evaluated:
                        continue
                    # Remove the cost of the crossing if the edge is connected to the node which has changed position in the current solution
                    if (u1 < u2 and cur_position[v1] > cur_position[v2]) or \
                       (u1 > u2 and cur_position[v1] < cur_position[v2]):
                        delta_cost -= w1 + w2
                        
                    # Add the cost of the crossing if the edge is connected to the node which has changed position in the new solution
                    if (u1 < u2 and new_position[v1] > new_position[v2]) or \
                         (u1 > u2 and new_position[v1] < new_position[v2]):
                          delta_cost += w1 + w2
        
        calculated_cost = current_cost + delta_cost
        
        # # Check if the calculated cost is correct
        # real_cost = 0
        # for (u1, v1, w1), (u2, v2, w2) in itertools.combinations(self.graph.edges, 2):
        #     if (u1 < u2 and new_position[v1] > new_position[v2]) or \
        #        (u1 > u2 and new_position[v1] < new_position[v2]):
        #         real_cost += w1 + w2
        #         
        # if calculated_cost != real_cost:
        #     print("Error")
        #     print("Calculated cost:", calculated_cost)
        #     print("Real cost:", real_cost)
        
        return calculated_cost
                    

In [65]:
graph, ordering = load_and_order_instance('in.txt')

vnd = ImprovedVND(
    graph=graph,
    initial_solution=ordering,
    step_function=StepFunction.BEST_IMPROVEMENT,
    max_iter=400,
    max_no_improve=200,
    neighborhood_order=[
        NeighborhoodType.SWAP,
        NeighborhoodType.INSERT,
        NeighborhoodType.REVERSE
    ]
)

best_solution, best_cost, stats = vnd.vnd_search()

# Print results and statistics
print(f"Best cost: {best_cost}")
print(f"Runtime: {stats.runtime:.2f} seconds")
print(f"Improvements per neighborhood:")
for n, impr in stats.improvements_per_neighborhood.items():
    print(f"  {n.value}: {impr}")
    

graph, ordering = load_and_order_instance('in.txt')

vnd = DeltaImprovedVND(
    graph=graph,
    initial_solution=ordering,
    step_function=StepFunction.BEST_IMPROVEMENT,
    max_iter=400,
    max_no_improve=200,
    neighborhood_order=[
        NeighborhoodType.SWAP,
        NeighborhoodType.INSERT,
        NeighborhoodType.REVERSE
    ]
)


best_solution, best_cost, stats = vnd.vnd_search()

# Print results and statistics
print(f"(Delta)Best cost: {best_cost}")
print(f"(Delta)Runtime: {stats.runtime:.2f} seconds")
print(f"(Delta)Improvements per neighborhood:")
for n, impr in stats.improvements_per_neighborhood.items():
    print(f"  {n.value}: {impr}")

Best cost: 80230.0
Runtime: 20.13 seconds
Improvements per neighborhood:
  swap: 115
  insert: 8
  reverse: 0
(Delta)Best cost: 80230.0
(Delta)Runtime: 15.49 seconds
(Delta)Improvements per neighborhood:
  swap: 115
  insert: 8
  reverse: 0
